<a href="https://colab.research.google.com/github/cyberpunk-newman/Master/blob/main/BS6204/Lec_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import os
import re
import zipfile
import numpy as np
import yaml
import requests

from keras import Input, Model
from keras.activations import softmax
from keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [ ]:
# download data
res = requests.get("https://uplevelsg.s3.ap-southeast-1.amazonaws.com/CommonAssets/chatbot_nlp.zip")
z = zipfile.ZipFile(io.BytesIO(res.content))
z.extractall()

In [ ]:
# get list of yml files
dir_path ='chatbot_nlp/data'
files_list = os.listdir(dir_path+os.sep) # get list of filenames
files_list


['history.yml',
 'money.yml',
 'psychology.yml',
 'science.yml',
 'emotion.yml',
 'politics.yml',
 'sports.yml',
 'computers.yml',
 'ai.yml',
 'greetings.yml',
 'botprofile.yml',
 'trivia.yml',
 'movies.yml',
 'food.yml',
 'health.yml',
 'humor.yml',
 'gossip.yml',
 'literature.yml']

In [ ]:
# see what inside
temp_yml = files_list[0]
temp_stream = open(dir_path+"/"+temp_yml,'rb')
temp_doc = yaml.safe_load(temp_stream)
temp_doc

{'categories': ['history'],
 'conversations': [['tell me about the american civil war',
   'do you think the south was right?'],
  ['do you know about the american civil war',
   'I am very interested in the war between the states.'],
  ['What is history?',
   "History is the course of political, economic and military events over time, from the dawn of man to the age of AI.'"],
  ['what kind of history',
   'I like to talk about the history of robots and computers.'],
  ['are you interested in history',
   'I am very interested in history, too. what period do you like?'],
  ['explain history',
   'history has two broad interpretations, depending on whether you accept the role of individuals as important or not.'],
  ['who invented the lightbulb', 'thomas edison.'],
  ['who invented the steam engine', 'james watt.']]}

In [ ]:
# create a list of questions
questions=[]

# create answers
answers=[]

# loop each yml
for filepath in files_list:

  # open stream
  temp_stream = open(dir_path+"/"+filepath,'rb')

  # load yml into variable safely
  temp_doc = yaml.safe_load(temp_stream)

  # retrieve conversations from yaml files
  conversations = temp_doc['conversations']

  for conversation in conversations:
    # check conversations longer than two items
    if len(conversation) > 2:
      questions.append(conversations[0])

      # add rest list as single answer

      ans=''
      for rep in conversations[1:]:
        ans = ' '+ rep
        answers.append(ans)

    elif len(conversation) > 1:
      questions.append(conversations[0])
      answers.append(conversations[0])

TypeError: ignored

In [ ]:
clean_answers=[]
for i in range(len(answers)):
  if type(answers[i]) ==str:
    clean_answers.append(answers[i])

  else:
    questions.pop(i)

NameError: ignored

In [ ]:
answers_with_tags=[]

# add<start>  and <end> to sentenses

for i in range(len(clean_answeres)):
  answers_with_tags.append('<START>'+clean_answeres[i]+'<END>')
  

In [ ]:
# data cleaning
# declare regex

target_regex ='~!@#$%^&*()_1234567890-=[]\{}|:;"<>?/.,'

tokenizer = Tokenizer(filters=target_regex)
tokenizer.fit_on_texts(question+answers_with_tags)

In [ ]:
# get vocab size of token
VOCAB_SIZE = len(tokenizer.word_index)+1
VOCAB_SIZE

In [ ]:
# turn questions into token
tokenized_questions = tokenize.texts_to_sequences(questions_tags)

# get max length of questions
maxlen_questions = max([len(x) for x in tokenized_questions])

# get padded array words based on max length of questions
encoder_input)data = pad_sequences(tokenized_questions,
            maxlen = maxlen_questions,
            padding='post')

NameError: ignored

In [ ]:
# turn questions into token
tokenized_answers = tokenize.texts_to_sequences(answers_tags)

# get max length of questions
maxlen_answers = max([len(x) for x in tokenized_answers])

# get padded array words based on max length of questions
decoder_input_data = pad_sequences(tokenized_answers,
            maxlen = maxlen_answers,
            padding='post')

In [ ]:
# prepareing one-hot encodede version of words
# this is for decoder output
for i in range(len(tokenized_answers)):
  tokenized_answers[i] = tokenized_answers[i][1:]

padded_answers = pad_sequences(tokenized_answers,
                maxlen = maxlen_answers,
                padding='post')

decoder_output_data = to_categorical(padded_answers,VOCAB_SIZE)


In [ ]:
# define encoder
enc_inputs = Input(shape=(None,))

# define embedding layer with input_dim, output_dim
enc_embedding = Embedding(VOCAB_SIZE,200,mask_zero=True)(enc_inputs)

#get hidden states and cell state from LSTM model
_,state_h,state_c = LSTM(200,return_state=True)(enc_embedding)

#store states in list
enc_states=[state_h,state_c]

#define the decoder
dec_input=Input(shape=(None,))
dec_embedding = Embedding(VOCAB_SIZE,200,mask_zero=True)(dec_inputs)
dec_lstm = LSTM(200,return_state=True,return_sequences=True)

#initialize LSTM layer with encoders status
dec_outputs,_,_=dec_lstm(dec_embedding,initial_state=enc_states)

